In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import missingno as msno

# Load Gold datasets
gold = pd.read_csv('/content/gold_updated_dataset.csv')
#gold['Year'] = pd.to_datetime(gold['Year'])


In [2]:
gold.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751 entries, 0 to 750
Data columns (total 55 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   typeCode                  751 non-null    object 
 1   freqCode                  751 non-null    object 
 2   refPeriodId               751 non-null    int64  
 3   Year                      751 non-null    int64  
 4   refMonth                  751 non-null    int64  
 5   Year_Check                751 non-null    int64  
 6   Country_code              751 non-null    int64  
 7   Country_ISO               751 non-null    object 
 8   Country                   751 non-null    object 
 9   flowCode                  751 non-null    object 
 10  flowDesc                  751 non-null    object 
 11  partnerCode               751 non-null    int64  
 12  partnerISO                751 non-null    object 
 13  partnerDesc               751 non-null    object 
 14  partner2Co

In [ ]:

import plotly.express as px

# Visualization : Faceted Plotly bar chart ---
fig = px.bar(
    gold,
    x='Country',
    y='TradeValue_USD',
    color='Country',                # Optional: gives each country a unique color
    facet_col='Year',               # Creates a separate subplot for each year
    facet_col_wrap=3,               # Wraps to a new row after 3 years (2 rows total for 2019-2024)
    title="Gold Trade by Country (2019-2024)"
)

# Improves readability by removing "Year=" from the subplot titles
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.show()




In [ ]:
import plotly.express as px

# Filter the data for the specific years
gold_filtered = gold[(gold['Year'] >= 2019) & (gold['Year'] <= 2024)]

# Sort the years to ensure they appear in order
years = sorted(gold_filtered['Year'].unique())

# Loop through each year and create a separate plot
for year in years:
    # Filter the dataframe for the specific year
    year_data = gold_filtered[gold_filtered['Year'] == year].sort_values(by='TradeValue_USD', ascending=False)

    # Create the individual bar chart
    fig = px.bar(
        year_data,
        x='Country',
        y='TradeValue_USD',
        color='Country',
        title=f"Gold Trade by Country ({year})",
        labels={'TradeValue_USD': 'Trade Value (USD)', 'Country': 'Country'}
    )

    
    fig.update_layout(showlegend=False)

    # Show the plot
    fig.show()

In [14]:
for year in range(2019, 2025):
    # Filter for the year, sort by value, and take top 3
    top_year = (gold[gold['Year'] == year]
                .groupby('Country')['TradeValue_USD']
                .sum()
                .sort_values(ascending=False)
                .head(5))

    print(f"\n--- Top 5 in {year} ---")
    for rank, (country, value) in enumerate(top_year.items(), 1):
        print(f"{rank}. {country} (${value:,.0f})")


--- Top 5 in 2019 ---
1. Switzerland ($61,907,448,388)
2. China, Hong Kong SAR ($25,192,103,911)
3. United Kingdom ($23,322,231,297)
4. United Arab Emirates ($19,113,230,537)
5. USA ($17,173,208,199)

--- Top 5 in 2020 ---
1. Switzerland ($71,656,680,445)
2. China, Hong Kong SAR ($41,283,352,728)
3. United Arab Emirates ($29,303,479,807)
4. United Kingdom ($21,447,642,822)
5. USA ($20,561,622,698)

--- Top 5 in 2021 ---
1. Switzerland ($86,757,610,494)
2. United Kingdom ($41,432,361,733)
3. United Arab Emirates ($33,786,922,895)
4. China, Hong Kong SAR ($30,801,993,999)
5. USA ($27,719,367,195)

--- Top 5 in 2022 ---
1. Switzerland ($100,227,921,556)
2. United Kingdom ($72,701,171,738)
3. USA ($37,175,292,755)
4. United Arab Emirates ($35,626,328,590)
5. China, Hong Kong SAR ($30,126,268,461)

--- Top 5 in 2023 ---
1. Switzerland ($107,165,596,657)
2. United Kingdom ($65,218,437,180)
3. United Arab Emirates ($52,829,324,884)
4. China, Hong Kong SAR ($40,235,768,234)
5. USA ($26,042,73

In [ ]:
import pandas as pd
import plotly.express as px

# Identify the Top 5 countries by total expenditure
top_3_countries = (
    gold.groupby('Country')['TradeValue_USD']
    .sum()
    .sort_values(ascending=False)
    .head(5)   # actually Top 5
    .index
    .tolist()
)

# Filter the dataframe
df_top3 = gold[gold['Country'].isin(top_3_countries)]

# PLOT (unchanged logic, fixed title)
fig = px.line(
    df_top3,
    x='Year',
    y='TradeValue_USD',
    color='Country',
    markers=True,
    title="Gold Expenditure Trend: Top 5 Countries (2019–2024)",
    labels={'TradeValue_USD': 'Expenditure (USD)', 'Year': 'Year'},
    template="plotly_white"
)

fig.update_layout(
    xaxis=dict(tickmode='linear'),
    hovermode="x unified"
)

fig.show()

# AGGREGATE YEARLY EXPENDITURE
yearly_expenditure = (
    df_top3
    .groupby(['Year', 'Country'], as_index=False)['TradeValue_USD']
    .sum()
)

# YEAR-WISE HIGHEST → LOWEST PRINT WITH GAPS
yearwise_rank_exp = (
    yearly_expenditure
    .fillna(0)
    .sort_values(['Year', 'TradeValue_USD'], ascending=[True, False])
)

# Clean numbers for display
yearwise_rank_exp['TradeValue_USD'] = (
    yearwise_rank_exp['TradeValue_USD']
    .round(0)
    .astype(int)
)

print("\nYEAR-WISE GOLD EXPENDITURE (USD)")
print("Countries ranked from Highest → Lowest\n")

for year, group in yearwise_rank_exp.groupby('Year'):
    print(f"YEAR {year}")
    print("-" * 55)

    group = group.reset_index(drop=True)
    group.index += 1

    print(
        group[['Country', 'TradeValue_USD']]
        .rename_axis('Rank')
        .reset_index()
        .to_string(index=False)
    )

    print("\n")



YEAR-WISE GOLD EXPENDITURE (USD)
Countries ranked from Highest → Lowest

YEAR 2019
-------------------------------------------------------
 Rank              Country  TradeValue_USD
    1          Switzerland     61907448388
    2 China, Hong Kong SAR     25192103911
    3       United Kingdom     23322231297
    4 United Arab Emirates     19113230537
    5                  USA     17173208199


YEAR 2020
-------------------------------------------------------
 Rank              Country  TradeValue_USD
    1          Switzerland     71656680445
    2 China, Hong Kong SAR     41283352728
    3 United Arab Emirates     29303479807
    4       United Kingdom     21447642822
    5                  USA     20561622698


YEAR 2021
-------------------------------------------------------
 Rank              Country  TradeValue_USD
    1          Switzerland     86757610494
    2       United Kingdom     41432361733
    3 United Arab Emirates     33786922895
    4 China, Hong Kong SAR     30801

In [ ]:
import pandas as pd
import plotly.express as px

# Define the Groups
BRICS = ["Brazil", "Russian Federation", "India", "China", "South Africa"]
US = ["USA"]
EU = ["Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark",
      "Estonia","Finland","France","Germany","Greece","Hungary","Ireland","Italy",
      "Latvia","Lithuania","Luxembourg","Malta","Netherlands","Poland","Portugal",
      "Romania","Slovakia","Slovenia","Spain","Sweden"]

# Assign Group
def assign_group(country):
    if country in BRICS:
        return 'BRICS'
    if country in US:
        return 'US'
    if country in EU:
        return 'EU'
    return 'Other'

gold['Group'] = gold['Country'].apply(assign_group)

# Filter & Aggregate by Year and Group
df_groups = gold[gold['Group'].isin(['BRICS', 'US', 'EU'])]

group_summary = (
    df_groups
    .groupby(['Year', 'Group'], as_index=False)['TradeValue_USD']
    .sum()
)

# YEAR-WISE HIGHEST → LOWEST PRINT WITH GAPS
yearwise_group_rank = (
    group_summary
    .fillna(0)
    .sort_values(['Year', 'TradeValue_USD'], ascending=[True, False])
)

# Clean numbers
yearwise_group_rank['TradeValue_USD'] = (
    yearwise_group_rank['TradeValue_USD']
    .round(0)
    .astype(int)
)

print("\nYEAR-WISE GOLD EXPENDITURE (USD): BRICS vs US vs EU")
print("Groups ranked from Highest → Lowest\n")

for year, group in yearwise_group_rank.groupby('Year'):
    print(f"YEAR {year}")
    print("-" * 55)

    group = group.reset_index(drop=True)
    group.index += 1

    print(
        group[['Group', 'TradeValue_USD']]
        .rename_axis('Rank')
        .reset_index()
        .to_string(index=False)
    )

    print("\n")   # gap between years

# PLOT (unchanged logic)
fig = px.line(
    group_summary,
    x='Year',
    y='TradeValue_USD',
    color='Group',
    markers=True,
    title="Gold Expenditure Trend: BRICS vs US vs EU (2019–2024)",
    labels={'TradeValue_USD': 'Total Expenditure (USD)', 'Year': 'Year'},
    template="plotly_white",
    color_discrete_map={
        'BRICS': '#E41A1C',
        'US': '#377EB8',
        'EU': '#4DAF4A'
    }
)

fig.update_layout(
    xaxis=dict(tickmode='linear'),
    hovermode="x unified"
)

fig.show()



YEAR-WISE GOLD EXPENDITURE (USD): BRICS vs US vs EU
Groups ranked from Highest → Lowest

YEAR 2019
-------------------------------------------------------
 Rank Group  TradeValue_USD
    1    EU     18170013261
    2    US     17173208199
    3 BRICS     15127612068


YEAR 2020
-------------------------------------------------------
 Rank Group  TradeValue_USD
    1 BRICS     33957241517
    2    EU     23223350867
    3    US     20561622698


YEAR 2021
-------------------------------------------------------
 Rank Group  TradeValue_USD
    1 BRICS     33115953776
    2    US     27719367195
    3    EU     19152422782


YEAR 2022
-------------------------------------------------------
 Rank Group  TradeValue_USD
    1    US     37175292755
    2    EU     21291544369
    3 BRICS     13529050408


YEAR 2023
-------------------------------------------------------
 Rank Group  TradeValue_USD
    1    US     26042732408
    2    EU     24990907406
    3 BRICS     13742552042


YEAR 2024


In [21]:

country_net = (
    gold
    .groupby("Country")["NetWeight_kg"]
    .sum()
    .sort_values(ascending=False)
)

print(country_net.head(10))

top_10 = country_net.head(10)

# Filter for the years 2019 to 2024
df_filtered = gold[(gold['Year'] >= 2019) & (gold['Year'] <= 2024)]

# Aggregate the NetWeight_kg by Country
country_net = df_filtered.groupby('Country')['NetWeight_kg'].sum().sort_values(ascending=False)

# Convert the Series to a DataFrame so 'Country' becomes a proper column
top_10_df = top_10.reset_index()

fig = px.bar(
    top_10_df,
    x="Country",
    y="NetWeight_kg",
    color="Country",
    title="Top 10 Countries by Increase in Gold Reserves (2019-2024)",
    labels={
        "NetWeight_kg": "Net Gold Increase (kg)",
        "Country": "Country"
    }
)

fig.show()



Country
Switzerland             7489847.184
United Kingdom          4807723.126
United Arab Emirates    3830266.299
China, Hong Kong SAR    3753220.044
USA                     2707033.766
Peru                    1977884.322
Canada                  1632133.097
Singapore               1303771.987
Australia               1294796.454
Argentina               1232030.440
Name: NetWeight_kg, dtype: float64


In [25]:

# Filter for the years 2019 to 2024
df_filtered = gold[(gold['Year'] == 2024)]

# Aggregate the NetWeight_kg by Country
country_net = df_filtered.groupby('Country')['NetWeight_kg'].sum().sort_values(ascending=False)

# Get the top 10 countries
top_10_df = country_net.head(10).reset_index()
print(top_10_df)
# Create the bar chart
fig = px.bar(
    top_10_df,
    x="Country",
    y="NetWeight_kg",
    color="Country",
    title="Top 10 Countries by Total Net Weight (2024)",
    labels={
        "NetWeight_kg": "Total Net Weight (kg)",
        "Country": "Country"
    }
)

# Display the plot
fig.show()

                Country  NetWeight_kg
0           Switzerland   1563791.414
1        United Kingdom    867029.730
2  China, Hong Kong SAR    742483.000
3                   USA    397850.788
4                Canada    374171.422
5             Australia    320334.930
6             Singapore    238025.989
7                 Japan    228849.430
8               Germany    219355.555
9             Argentina    165757.940


In [9]:
import plotly.express as px

# Group by Year and Country, sum NetWeight_kg
country_net = (
    gold
    .groupby(['Year', 'Country'])['NetWeight_kg']
    .sum()
    .reset_index()
)

# List of years in your dataset
years = sorted(gold['Year'].unique())

# Loop through each year and plot top 10 countries
for year in years:
    top_10_df = country_net[country_net['Year'] == year].sort_values(
        by='NetWeight_kg', ascending=False
    ).head(10)

    fig = px.bar(
        top_10_df,
        x="Country",
        y="NetWeight_kg",
        color="Country",
        title=f"Top 10 Countries by Net Gold (kg) in {year}",
        labels={
            "NetWeight_kg": "Net Gold (kg)",
            "Country": "Country"
        }
    )
    fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Make sure country_net exists
# Group by Country across all years to get total NetWeight
country_net = gold.groupby("Country")["NetWeight_kg"].sum().sort_values(ascending=False)

# Get top 8 countries
top_8_countries = country_net.index[:8].tolist()   # Top 8

# Filter data for these countries and sum by Year
trend_top8 = (
    gold[gold["Country"].isin(top_8_countries)]
    .groupby(["Year", "Country"], as_index=False)["NetWeight_kg"]
    .sum()
)

# PLOT
fig = px.line(
    trend_top8,
    x="Year",
    y="NetWeight_kg",
    color="Country",
    markers=True,
    title="Gold Reserve Trends for Top 8 Countries",
    labels={"NetWeight_kg": "Net Gold Change (kg)", "Year": "Year"}
)

fig.update_layout(
    xaxis=dict(tickmode="linear"),
    hovermode="x unified"
)

fig.show()

# YEAR-WISE HIGHEST → LOWEST PRINT
yearwise_rank = trend_top8.sort_values(["Year", "NetWeight_kg"], ascending=[True, False])

# Round numbers for clean display
yearwise_rank["NetWeight_kg"] = yearwise_rank["NetWeight_kg"].round(0).astype(int)

print("\nYEAR-WISE GOLD NET WEIGHT (kg)")
print("Countries ranked from Highest → Lowest\n")

for year, group in yearwise_rank.groupby("Year"):
    print(f"YEAR {year}")
    print("-" * 45)

    group = group.reset_index(drop=True)
    group.index += 1  # Rank starting at 1

    print(
        group[["Country", "NetWeight_kg"]]
        .rename_axis("Rank")
        .reset_index()
        .to_string(index=False)
    )

    print("\n")



YEAR-WISE GOLD NET WEIGHT (kg)
Countries ranked from Highest → Lowest

YEAR 2019
---------------------------------------------
 Rank              Country  NetWeight_kg
    1          Switzerland       1401718
    2 China, Hong Kong SAR        575125
    3       United Kingdom        531505
    4                 Peru        395439
    5                  USA        391630
    6            Singapore        273823
    7               Canada             0
    8 United Arab Emirates             0


YEAR 2020
---------------------------------------------
 Rank              Country  NetWeight_kg
    1          Switzerland       1281896
    2 United Arab Emirates       1143173
    3 China, Hong Kong SAR        725788
    4       United Kingdom        377592
    5                  USA        367836
    6                 Peru        348421
    7            Singapore        297682
    8               Canada        290000


YEAR 2021
---------------------------------------------
 Rank             

In [ ]:
# Define the Groups
BRICS = ["Brazil", "Russian Federation", "India", "China", "South Africa"]
US = ["USA"]
EU = ["Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark",
      "Estonia","Finland","France","Germany","Greece","Hungary","Ireland","Italy",
      "Latvia","Lithuania","Luxembourg","Malta","Netherlands","Poland","Portugal",
      "Romania","Slovakia","Slovenia","Spain","Sweden"]

# Assign group
def assign_group(country):
    if country in BRICS: return 'BRICS'
    if country in US: return 'US'
    if country in EU: return 'EU'
    return 'Other'

gold['Group'] = gold['Country'].apply(assign_group)

# Filter and aggregate
df_groups = gold[gold['Group'].isin(['BRICS', 'US', 'EU'])]

group_summary = (
    df_groups
    .groupby(['Year', 'Group'])['NetWeight_kg']
    .sum()
    .reset_index()
)

# Print year-wise net gold change
for year in sorted(group_summary['Year'].unique()):
    print(f"\n--- {year} Net Gold Change ---")
    year_data = group_summary[group_summary['Year'] == year] \
        .sort_values(by='NetWeight_kg', ascending=False)
    for _, row in year_data.iterrows():
        print(f"{row['Group']}: {row['NetWeight_kg']:,.0f} kg")

# Plot
fig = px.line(
    group_summary,
    x='Year',
    y='NetWeight_kg',
    color='Group',
    markers=True,
    title="Net Gold Reserve Change: BRICS vs US vs EU (2019–2024)",
    labels={'NetWeight_kg': 'Net Gold Change (kg)', 'Year': 'Year'},
    template="plotly_white",
    color_discrete_map={'BRICS': '#E41A1C', 'US': '#377EB8', 'EU': '#4DAF4A'}
)

fig.update_layout(xaxis=dict(tickmode='linear'), hovermode="x unified")
fig.show()



--- 2019 Net Gold Change ---
US: 391,630 kg
BRICS: 253,145 kg
EU: 240,902 kg

--- 2020 Net Gold Change ---
BRICS: 382,032 kg
US: 367,836 kg
EU: 241,644 kg

--- 2021 Net Gold Change ---
US: 481,773 kg
BRICS: 459,610 kg
EU: 182,409 kg

--- 2022 Net Gold Change ---
US: 650,243 kg
EU: 290,631 kg
BRICS: 56,668 kg

--- 2023 Net Gold Change ---
US: 417,701 kg
EU: 243,005 kg
BRICS: 62,737 kg

--- 2024 Net Gold Change ---
US: 397,851 kg
EU: 281,869 kg
BRICS: 159,226 kg


In [ ]:
# Filter out rows with zero or missing weight to avoid division errors
gold_filtered = gold[gold['NetWeight_kg'] > 0].dropna(subset=['TradeValue_USD', 'NetWeight_kg'])

# Aggregate Data by Year and Flow (Export/Import)
# sum values and weights first to get an accurate global annual price
trend_data = gold_filtered.groupby(['Year', 'flowDesc']).agg({
    'TradeValue_USD': 'sum',
    'NetWeight_kg': 'sum'
}).reset_index()

# Calculate Price per kg (USD/kg)
trend_data['Price_per_kg'] = trend_data['TradeValue_USD'] / trend_data['NetWeight_kg']

print("gold Price Trend Data (2019-2024):")
print(trend_data[['Year', 'Price_per_kg']].to_string(index=False))

# Create Plotly Line Chart
fig = px.line(
    trend_data,
    x='Year',
    y='Price_per_kg',
    color='flowDesc',
    title='Global Gold Price Trend (2019 - 2024)',
    labels={
        'Price_per_kg': 'Price (USD per kg)',
        'Year': 'Year',
        'flowDesc': 'Trade Flow'
    },
    markers=True,
    template='plotly_white'
)

# Customize the layout
fig.update_layout(hovermode='x unified')

# Display the chart
fig.show()

gold Price Trend Data (2019-2024):
 Year  Price_per_kg
 2019  37387.715553
 2020  45596.072866
 2021  50949.423664
 2022  50389.019952
 2023  41938.876304
 2024  71395.460291


In [ ]:

# Define the Groups
BRICS = ["Brazil", "Russian Federation", "India", "China", "South Africa"]
US = ["USA"]
EU = ["Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark",
      "Estonia","Finland","France","Germany","Greece","Hungary","Ireland","Italy",
      "Latvia","Lithuania","Luxembourg","Malta","Netherlands","Poland","Portugal",
      "Romania","Slovakia","Slovenia","Spain","Sweden"]

# Create a Group mapping column
def categorize(country):
    if country in BRICS: return 'BRICS'
    if country in US: return 'US'
    if country in EU: return 'EU'
    return 'Other'

gold['Category'] = gold['Country'].apply(categorize)

# Filter for years since 2021 and only the groups of interest
# use qty as the metric for "reserves/volume"
df_trends = gold[(gold['Year'] >= 2021) & (gold['Category'] != 'Other')]
df_yearly = df_trends.groupby(['Year', 'Category'])['qty'].sum().reset_index()

# Calculate CAGR (2021 to 2024)
start_year = 2021
end_year = df_yearly['Year'].max()
years_diff = end_year - start_year

cagr_data = []
for cat in df_yearly['Category'].unique():
    subset = df_yearly[df_yearly['Category'] == cat]
    try:
        v_start = subset[subset['Year'] == start_year]['qty'].values[0]
        v_end = subset[subset['Year'] == end_year]['qty'].values[0]
        cagr = (v_end / v_start)**(1 / years_diff) - 1
        cagr_data.append({'Category': cat, 'CAGR': f"{cagr:.2%}"})
    except IndexError:
        cagr_data.append({'Category': cat, 'CAGR': "Incomplete Data"})

# Print the CAGR results
print("--- CAGR Results (2021-2024) ---")
print(pd.DataFrame(cagr_data))

# Plot the Trend using Plotly Express
fig = px.line(
    df_yearly,
    x='Year',
    y='qty',
    color='Category',
    title='Gold Trade Volume Trend by Group (2021 - 2024)',
    labels={'qty': 'Total Weight (kg)', 'Year': 'Year'},
    markers=True,
    template='plotly_white'
)

fig.show()

--- CAGR Results (2021-2024) ---
  Category     CAGR
0    BRICS  -29.77%
1       EU   24.39%
2       US   -6.18%
